In [41]:
import os
import requests
import urllib.parse
import datetime
import pandas as pds
import json
import time

In [50]:
def datetime_converter(type, time):
    """
    Convert between date and timestamp.

    Parameters:
    type (str): 'date' to convert timestamp to date, 'timestamp' to convert date to timestamp.
    time (str/int): Time in either date format 'YYYY-MM-DD' or timestamp format in milliseconds.

    Returns:
    datetime or int: Converted date or timestamp.
    """
    if type == 'date':
        timestamp = int(time) / 1000
        date = datetime.datetime.fromtimestamp(timestamp)
        return date
    elif type == 'timestamp':
        date = datetime.datetime.strptime(time, '%Y-%m-%d')
        timestamp = int(date.timestamp())
        return timestamp
    else:
        raise ValueError("Invalid type. Supported types are 'date' and 'timestamp'.")

In [52]:
def get_api_key():
    """
    Retrieve the Riot API key from environment variables.

    Returns:
    str: API key.
    """
    with open('../api_key.txt', 'r') as file:
        api_key = file.read().replace('\n', '')
    if not api_key:
        raise ValueError("API key not found. Please set the RIOT_API_KEY environment variable.")
    return api_key

In [53]:
def get_puuid(game_name, name_tag):
    """
    Retrieve the PUUID for a given game name and tag.

    Parameters:
    game_name (str): Game name.
    name_tag (str): Game tag.

    Returns:
    str: PUUID.
    """
    api_key = get_api_key()
    url_puuid = f'https://europe.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{game_name}/{name_tag}?api_key={api_key}'
    response_puuid = requests.get(url_puuid)
    response_puuid.raise_for_status()
    return response_puuid.json()['puuid']

In [54]:
def get_matchids(puuid, amount_of_games=10000, starttime=0, endtime=0, type='ranked'):
    """
    Retrieve match IDs for a given PUUID.

    Parameters:
    puuid (str): PUUID of the player.
    amount_of_games (int): Number of games to fetch.
    starttime (int): Start time for the matches in timestamp.
    endtime (int): End time for the matches in timestamp.
    type (str): Type of matches (e.g., 'ranked').

    Returns:
    list: List of match IDs.
    """
    api_key = get_api_key()
    match_ids = []
    count = 0
    start = 0

    while count < amount_of_games:
        remaining_games = amount_of_games - count
        games_to_fetch = min(remaining_games, 100)
        params = {
            'type': type,
            'start': start,
            'count': games_to_fetch,
            'api_key': api_key
        }
        if starttime != 0:
            params['startTime'] = starttime
        if endtime != 0:
            params['endTime'] = endtime

        url_matchids = f'https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids'
        response_matchids = requests.get(url_matchids, params=params)
        
        if response_matchids.status_code == 429:  # Rate limiting
            retry_after = int(response_matchids.headers.get('Retry-After', 1))
            time.sleep(retry_after)
            continue

        response_matchids.raise_for_status()
        match_ids.extend(response_matchids.json())

        count += games_to_fetch
        start += games_to_fetch

    return match_ids

In [55]:
def get_match_data(match_id):
    """
    Retrieve match data for a given match ID.

    Parameters:
    match_id (str): Match ID.

    Returns:
    dict: Match data.
    """
    api_key = get_api_key()
    url_match = f'https://europe.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={api_key}'
    try:
        response_match = requests.get(url_match)
        response_match.raise_for_status()
        response_json = response_match.json()
        
        if 'status' in response_json and response_json['status']['status_code'] == 404:
            print('Data not found - match file not found')
            return None

        return response_json
    except requests.RequestException as e:
        print(f'Error occurred: {str(e)}')
        return {}

In [56]:
def filter_data(data, my_puuid, puuids_to_exclude):
    """
    Filter match data for specific conditions.

    Parameters:
    data (list): List of match data.
    my_puuid (str): PUUID of the player.
    puuids_to_exclude (list): List of PUUIDs to exclude.

    Returns:
    list: Filtered match data.
    """
    filtered_data = []
    for match in data:
        my_summoner = match['metadata']['participants'].index(my_puuid)
        team_id_my_summoner = match['info']['participants'][my_summoner]['teamId']
        for participant in match['info']['participants']:
            if (participant['teamPosition'] == 'BOTTOM' or participant['teamPosition'] == 'UTILITY') and participant['teamId'] == team_id_my_summoner and participant['puuid'] not in puuids_to_exclude:
                filtered_data.append(participant)
    return filtered_data

In [57]:
match_ids = get_matchids(get_puuid('Zp4t', 'EUW'))

['EUW1_6940254163',
 'EUW1_6940175791',
 'EUW1_6940076528',
 'EUW1_6939996749',
 'EUW1_6939935640',
 'EUW1_6939858531',
 'EUW1_6938037868',
 'EUW1_6937976552',
 'EUW1_6937874429',
 'EUW1_6937040811',
 'EUW1_6937027463',
 'EUW1_6936925362',
 'EUW1_6936862407',
 'EUW1_6936784872',
 'EUW1_6936700517',
 'EUW1_6936654597',
 'EUW1_6935544370',
 'EUW1_6934447572',
 'EUW1_6934394055',
 'EUW1_6931317539',
 'EUW1_6931197230',
 'EUW1_6931041325',
 'EUW1_6930989822',
 'EUW1_6930943754',
 'EUW1_6930235023',
 'EUW1_6929018498',
 'EUW1_6924406987',
 'EUW1_6924361135',
 'EUW1_6924328766',
 'EUW1_6921801705',
 'EUW1_6921772362',
 'EUW1_6921713287',
 'EUW1_6921640887',
 'EUW1_6921441131',
 'EUW1_6921348343',
 'EUW1_6920891682',
 'EUW1_6918548140',
 'EUW1_6918502374',
 'EUW1_6916346547',
 'EUW1_6916291954',
 'EUW1_6916201256',
 'EUW1_6914047346',
 'EUW1_6914021906',
 'EUW1_6913991531',
 'EUW1_6913582627',
 'EUW1_6913517111',
 'EUW1_6913398250',
 'EUW1_6913310666',
 'EUW1_6912085408',
 'EUW1_6912005040',


In [37]:
import json

# Load previously saved data
try:
    with open('match_data.json', 'r') as file:
        filtered_data = json.load(file)
except FileNotFoundError:
    filtered_data = []

my_puuid = get_puuid('Zp4t', 'EUW')
puuids_to_exclude = [get_puuid(puuid[0], puuid[1]) for puuid in [('Lurrvpöse', 'EUW'), ('Deadlystream538', 'EUW')]]

# Get the timestamp of the latest match, or None if no matches have been saved
latest_timestamp = None if not filtered_data else max(match['info']['game_datetime'] for match in filtered_data)

match_ids = get_matchids(my_puuid, amount_of_games=50, type='ranked', starttime=latest_timestamp)

for match in match_ids:
    match_data = get_match_data(match)
    if match_data is not None:
        filtered_data.extend(filter_data([match_data], my_puuid, puuids_to_exclude))

# Save the data to a file
with open('match_data.json', 'w') as file:
    json.dump(filtered_data, file)

KeyError: 'metadata'

In [34]:
filtered_data = pd.DataFrame(filtered_data)
kda = filtered_data[['kills', 'deaths', 'assists']]
kda['deaths'].replace(0, 1, inplace=True)
kda_mean = kda.mean()
print((kda_mean['kills'] + kda_mean['assists']) / kda_mean['deaths'])

KeyError: "None of [Index(['kills', 'deaths', 'assists'], dtype='object')] are in the [columns]"